In [1]:
import datetime
import os

from dateutil.relativedelta import relativedelta

import camelot
import dateparser
import magic
import pandas as pd
import requests
import tabula

from bs4 import BeautifulSoup

In [2]:
!pwd

/home/sean/Projects/housing/housing/notebooks


In [3]:
os.chdir('/home/sean/Projects/housing/housing/data/raw')
!ls

RAENewsRelease2-Feb2019.pdf  RAENewsRelease-June2018.pdf
RAENewsRelease-Apr2018.pdf   RAENewsRelease-Mar2018.pdf
RAENewsRelease-Apr2019.pdf   RAENewsRelease-Mar2019.pdf
RAENewsRelease-Aug2018.pdf   RAENewsRelease-May2018.pdf
RAENewsRelease-Dec2018.pdf   RAENewsRelease-May2019.pdf
RAENewsRelease-Feb2018.pdf   RAENewsRelease-Nov2018.pdf
RAENewsRelease-Jan2018.pdf   RAENewsRelease-Oct2018.pdf
RAENewsRelease-Jan2019.pdf   RAENewsRelease-Sept2018.pdf
RAENewsRelease-July2018.pdf  YEGmedia-RAEstatsMay2019.pdf


In [4]:
link_page = "https://realtorsofedmonton.com/web/RAE_Public/Market_Stats/Monthly%20Market%20Statistics/RAE_Public/Market_Statistics/Monthly_Market_Stats.aspx?hkey=ab27cbda-7c81-4393-8a5f-aaf1277eba97"
page = requests.get(link_page)
soup = BeautifulSoup(page.content, 'html.parser')

pdf_links = {link['href'] for link in soup.find_all('a')
             if ('href' in link.attrs
                 and 'RAE' in link['href']
                 and 'pdf' == link['href'][-3:])
            }

In [5]:
for pdf in pdf_links:
    print(pdf)
    #run once: !wget $pdf ./

https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-May2019.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Jan2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Jan2019.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-July2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Sept2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Oct2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Apr2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Apr2019.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Mar2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Nov2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-June2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Dec2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-May2018.pdf
https://member.ereb.com/WEB/Documents/pdf/RAENewsRelease-Aug2018.pdf
https://member.ereb.com/WEB/Doc

In [ ]:
pdfs = !ls
for pdf in pdfs:
    if 'HTML' in magic.from_file(pdf):
        pass
        #wget alternative format
 